# Visual Inspection  of Health Center-RRL and Referral Hospital RRL & Cardsheet  Clinical Data

This is a part of a Ph.D research project entitled with "Developing clincal decision support point of care instrument for low resource settings". It's supervised by  Bart Jansen (Prof), Frank Verbeke (Prof), Jan Cornelis (Prof), Girum Ketema (Ass.Prof) and Demesewu Amenu (Associate Prof). 
  

# Ehtiopian Primary Health System (Highlights) 

1. The public health  care system in Ethiopia is structured around the concept of a “health network model” that uses a three tiered health care delivery levels namely primary, secondary and tertiary levels with defined populations to be served at each level.  

2. For rural settings at the base is the primary health care unit that is a health centre with five satellite health posts catering to a population of 25,000; followed by  a primary hospital, serving a population of  60,000 to 100,000; and next a general  hospital, providing services to 1 up to 1.5 million beneficiaries.  For urban setting at the base is a health center serving 40,000 people, followed by a general hospital as in the rural setting. At the apex of both structures is specialized hospital which serves 3.5 to 5 million people


In [ ]:
#importing the required labraries 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import holoviews as hv
import datashader as ds
import dask as dask
import bokeh as bokeh
#import chart_studio.plotly as py
import holoviews as hv
import datashader as ds
import dask as dask
import bokeh as bokeh
import cufflinks as cf


import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
#init_notebook_mode(connected=True)
import ipywidgets as widgets
from ipywidgets import SelectionSlider
from ipywidgets import interact, interact_manual
import os
from IPython.display import Image
from PIL import ImageDraw 

# Reference file used to explore referral flow  

The diagram below is constucted based on the Federal Ministry of Health of Ethiopia referral network documents and guidlines. It aims to create or improve liaison services and that of the referral network so that effective  utilization of health care services resources to ensure provision of better quality of care. It will targets all liaison officers, health service providers, facility managers and other stake holders.

In [ ]:
fdir='images/'
#@interact
#def show_images(referenceFile=os.listdir(fdir)):
 #   display(Image(fdir+referenceFile,width=100, height=100)) 
Image(filename='referralFlow.png') 

In [ ]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 


original = Image.open("referralFlow.png")
draw = ImageDraw.Draw(original)
draw.text((100, 100),"Sample Text")
#original. save("with_text.png")

# Facts and challenges of the referral service 

1. Patient view the referral as a failure of the service, severe case, extra cost and so on. 
2. Matching the facility or the service based on the patients' condition is challenging. A need to deliver automated and evidence based service directory, matching, management, and readiness. 
3. Reduce delay for mitigating the complication of critical patient management e.g. cost, infrastructure, service, and transportation 
5. Getting feedback e.g. the health center is challenged to get feedback from hospital  

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('rsOeJFYro2Y', start=13)

# Q ? 

Inspecting the CGs, RRL and patient card-sheet to uncover:
1. How the referral decision is made. 
2. What cases are referred to the next level? 
3. Is it appropriate referral or not
4. pattern for exploring the referral flow and mathcing e.g.explore findings that point to more than one disease. We call it "conisderation path or crossroad PLUS(+) referral"
5. What are the missed feedback. e.g. compute agreggated matrix b/n RRL-out and RRL-in Form  

# Part- I: Health Center RRL

# Importing the CSV dataset

In [ ]:
#import health center RRL 
#hchcrrldfrrldata = pd.read_csv("HC RRL-Table 1.csv", encoding='utf-8')
#hcrrldf = pd.read_csv("JHC_RRL-1.csv", encoding='utf-8')
   
#import the health cetner RRL (wit ICPC-2 and ICD11 format)

hcrrldf = pd.read_csv("HC_ICD11_RRL-Table 1.csv", encoding='utf-8')


In [ ]:
hcrrldf.shape

In [ ]:
hcrrldf

In [ ]:
#hcrrldf['Date'] = hcrrldf['Date'].replace(['29/2/2010'],'28/2/2010')

In [ ]:
hcrrldf['Date']

In [ ]:
hcrrldf['Date'] = pd.to_datetime(hcrrldf['Date'], errors='coerce')
hcrrldf['Date'].value_counts().plot( marker='o', linestyle='-', figsize=(15, 5))

In [ ]:
start, end = '2/2/2010', '17/12/2011'
xx = hcrrldf['Date'].loc[start:end]
xx

In [ ]:
start, end = '2/2/2010', '17/12/2011'
hcrrldf['Date'] = pd.to_datetime(hcrrldf['Date'], errors='coerce')
fig, ax = plt.subplots()
ax.plot(hcrrldf['Date'].loc[start:end],
        marker='.', linestyle='-', linewidth=0.5, label=' Referal')
ax.set_ylabel('Solar Production (GWh)')
ax.legend();

In [ ]:
# Start and end of the date range to extract
start, end = '2/2/2010', '17/12/2011'
# Plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(hcrrldf.loc[start:end, 'Date'],
marker='.', linestyle='-', linewidth=0.5, label=' Referal')
ax.set_ylabel('Solar Production (GWh)')
ax.legend();

In [ ]:
#interacting with the raw healthcenter CSV dataset
@interact
def show_referral_by_age(column=["Age"],x=(5,100,5)):
    return hcrrldf.loc[hcrrldf[column] >x]

In [ ]:
hcrrldf['Reffering_Organization'].value_counts()

# Spliting Date into Day-Month-Year

In [ ]:
#Splitting the date into year, month and date
hcrrldf['Date'] = pd.to_datetime(hcrrldf.Date,errors='coerce')
hcrrldf['Day'],hcrrldf['Month'],hcrrldf['Year'] = hcrrldf.Date.dt.day, hcrrldf.Date.dt.month, hcrrldf.Date.dt.year

#display the new RRL list  
@interact
def show_referral_by_age(column=["Age","Year","Month"],x=(5,200,5)):
    return hcrrldf.loc[hcrrldf[column] >x]

#df.shape

# RRL  flitering by referral reason 

In [ ]:
# Pop-up the RRL by referral reason only .... filter function is required 

#Step-1: a generic fucntion is rquired to capture uniques unique values 
ALL = 'ALL' 
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

#Step-2: intialize a variable to capture the referral reason
reason4Referral = widgets.Dropdown(options =unique_sorted_values(hcrrldf.Reason_for_Referral_Trans),description="Ref. Reason")


referral_reason_output = widgets.Output()

#Step-3: Write an event-handeler function     
def referral_reason_eventhandler(change):
    referral_reason_output.clear_output() #the output didn't refersh when selecting a new referral reason rather it will append at the end
    with referral_reason_output:
        if (change.new == ALL):
            display(hcrrldf)
        else:
            display(hcrrldf[hcrrldf.Reason_for_Referral_Trans == change.new])
            
#Step-4 : Bind to the event handler to the dropdown value  
reason4Referral.observe(referral_reason_eventhandler, names='value')

# Step-5: Display the result
display(reason4Referral)
display(referral_reason_output)

# Finding the correlations between two columns 

In [ ]:
@interact 
def correlations(column1=list(hcrrldf.select_dtypes('number').columns),
                column2=list(hcrrldf.select_dtypes('number').columns)):
    print(f"Correlation: {hcrrldf[column1].corr(hcrrldf[column2])}")
    
    
#need of the developing co-relation matrix for each attributes 

 # RRL Data Visualization  

1. Visualization of RRL by Age

In [ ]:
rage = hcrrldf.Age
xp = rage.value_counts()
plt.title('Referral Registration Logbook', fontsize=14)
plt.xlabel('Age', fontsize=14)
plt.ylabel('Summary', fontsize=14)
xp.plot(kind='bar')

In [ ]:
# There is two typing eror values 
#Replace the average  Age  
avrAge=hcrrldf['Age'].mean()
#Replacing the two raws with an Average age value i.e. 2423 and 88
hcrrldf.Age.replace([2423,88],29, inplace=True)

In [ ]:
# The new distruption of Age is looks like follows
rage = hcrrldf.Age
xp = rage.value_counts()
plt.title('Referral Registration Logbook', fontsize=14)
plt.xlabel('Age', fontsize=14)
plt.ylabel('Summary', fontsize=14)
xp.plot(kind='bar')

In [ ]:
commute_times = hcrrldf.Age
bin_edges= [15,20,25,30,35,40,45,50]
plt.hist(commute_times,
         bins=bin_edges,
         density=False,
         histtype='bar',
         color='b',
         edgecolor='k',
         alpha=0.5)
plt.xlabel('Age')
plt.ylabel('Number')
plt.title('Figure 8 : Summary of Referral Age')


plt.show()

In [ ]:
commute_times.value_counts()

2. Interactive visulization: Inspecting numberic attributs at  the RRL dataset

In [ ]:
#building scatter plot for numeric values 
themes=cf.getThemes()

#exclude or replace the nan value in the feed back column 

hcrrldf['Feed_Back'] = hcrrldf['Feed_Back'].replace(np.nan, "Missed Value")

@interact
def scatter_plot(x=list(hcrrldf.select_dtypes('number').columns), 
                 y=list(hcrrldf.select_dtypes('number').columns)[1:],
                 #theme=list(cf.themes.THEMES.keys()), 
                 theme=list(cf.getThemes()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    
    hcrrldf.iplot(kind='scatter', x=x, y=y, mode='markers', 
             xTitle=x.title(), yTitle=y.title(), 
             text='DiseaseName',
             title=f'{y.title()} vs {x.title()}',
            theme=theme, colorscale=colorscale)



3. Interactive visulization: Inspecting objects or catagorical attributes at  the RRL dataset using graphs

In [ ]:
def is_categorical(Referring_Organization):
    return Referring_Organization.dtype.name == 'category'

 #hcrrldf.Referring_Organization.dtype.name

In [ ]:
#building Histogram for categorical values or objects 
@interact 
def hist(x=list(hcrrldf.select_dtypes('object').columns),
                 #theme=list(cf.themes.THEMES.keys()), 
                 theme=list(cf.getThemes()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    hcrrldf[x].iplot(kind='hist', x=x, mode='markers', xTitle='x',
                   yTitle='count', title='Distribution',
                   theme=theme, colorscale=colorscale)

4. Exploring the frequencey based on certain attributes using crosstab. This helps to visulize the frequencey of the flow of refferal based on the two selected data attributes.

In [ ]:
#Implmentaitng cross tab between two variables 
@interact 
def crosstab(x=list(hcrrldf.select_dtypes('object').columns), 
             y=list(hcrrldf.select_dtypes('object').columns)[1:],
             theme=list(cf.getThemes()), 
             colorscale=list(cf.colors._scales_names.keys())):
    pd.crosstab(hcrrldf[x],hcrrldf[y]).plot.barh().legend(loc='center left', bbox_to_anchor=(0.3, 0.5))


In [ ]:
pd.crosstab(hcrrldf['Reason_for_Referral_Trans'],hcrrldf['Refered_to']).plot.barh().legend(loc='center left', bbox_to_anchor=(0.3, 0.5))
plt.xlabel('Number of Cases')
plt.ylabel('Referral Reason')

In [ ]:
hcrrldf['Reason_for_Referral_Trans'].value_counts(normalize=True)*100

5. Exploring the frequencey or the summary of based on a certain attributes using catplot. This helps to visulize the frequencey of the flow of refferal based on the three selected data attributes.

In [ ]:
# Visulization of referred reason and age category based on the referred orginazation 
# e.g. x= Refered_to, Y=age and Z= reason_for referral_trans
# And also different combination of VIZ also explored 
@interact 
def catplot(x=list(hcrrldf.select_dtypes('object').columns), 
            y=list(hcrrldf.select_dtypes('number').columns)[1:],
            z=list(hcrrldf.select_dtypes('object').columns)[2:],
            theme=list(cf.getThemes()), 
            colorscale=list(cf.colors._scales_names.keys())):
    sns.catplot(x=x, y=y, hue=z,aspect=3,kind="swarm",  data=hcrrldf);

In [ ]:
@interact 
def catplot(
    style="ticks", 
    color_codes=True,
    height=widgets.IntSlider(min=3, max=30, step=1, value=6), 
    aspect=widgets.IntSlider(min=1, max=30, step=1, value=2)):
    sns.catplot(x="Age", y="Refered_to", hue="Reason_for_Referral_Trans",height=height, aspect=aspect,
                kind="swarm",orient="h",palette="Set3", data=hcrrldf);


In [ ]:
# Here is important for analysis: The summary of Age, Referral reason and Referred hospital
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

g=sns.catplot(x="Age", y="Refered_to", hue="Reason_for_Referral_Trans",kind="swarm",orient="h",height=6, aspect=2, palette="Set3",  data=hcrrldf);
plt.xlabel('Age')
plt.ylabel('Referral to')
legend = g._legend
legend.set_title("Reason for referral")

6. Analysis of the disease after  aligning with ICPC-2 and ICD-11 classfication

In [ ]:
# Here is important for analysis: The summary of Age, Referral reason and Referred hospital
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

sns.catplot(x="Age", y="ICPC2_Classfication", hue="Reason_for_Referral_Trans",kind="swarm",orient="h",height=6, aspect=2, palette="Set3",  data=hcrrldf);

In [ ]:
# Here is important for analysis: The summary of Age, Referral reason and Referred hospital
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

g=sns.catplot(x="Age", y="ICD11_Classfication", hue="Reason_for_Referral_Trans",kind="swarm",orient="h",height=6, aspect=2, palette="Set3",  data=hcrrldf);
plt.xlabel('Age')
plt.ylabel('Referral Cases (ICD 11 Classification Format)')
legend = g._legend
legend.set_title("Referral Reason")

# Part -II:  Hospital RRL & Cardsheet: the dataset is collected randomly    

1. Importing  the referral data 

In [ ]:
#jurr2data = pd.read_csv("Compiled_from_JUSH-Table 1.csv", encoding='utf-8')
#jurr2data = pd.read_csv("Compiled_from_JUSH_1.csv", encoding='utf-8')

# Hospital RRL and cardsheet  ... as per the classifcication of  ICPC2 and ICD11 
#import the referred hospitpal RRL and patient cardsheet (with ICPC-2 and ICD11 format)
jurr2data = pd.read_csv("FullHospRRL-Table 1.csv", encoding='utf-8')



In [ ]:
jurr2data.shape

In [ ]:
jurr2data.columns

2. Splitting referral date in to Year, Month and Day

In [ ]:
#splitting the date of arrival in the hospital into Year, Month and day
#Splitting the date into year, month and date
jurr2data['Reffered_Date'] = pd.to_datetime(jurr2data.Reffered_Date,errors='coerce')
jurr2data['Day'],jurr2data['Month'],jurr2data['Year'] = jurr2data.Reffered_Date.dt.day, jurr2data.Reffered_Date.dt.month, jurr2data.Reffered_Date.dt.year

#display the new RRL list  
#@interact
#def show_referral_by_age(column=["Year","Month"],x=(10,200,10)):
 #   return jurr2data.loc[jurr2data[column] >x]

In [ ]:
jurr2data.head(1)

3. Flitering by referral reason 

3.1. Filter by Referral Reason only

In [ ]:
# Pop-up the RRL by referral reason only .... filter function is required 

#Step-1: a generic fucntion is rquired to capture uniques unique values 
ALL = 'ALL' 
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

#Step-2: intialize a variable to capture the referral reason
reason4Referral = widgets.Dropdown(options =unique_sorted_values(jurr2data.HC_Referral_Reason),description="Ref. Reason")


referral_reason_output = widgets.Output()

#Step-3: Write an event-handeler function     
def referral_reason_eventhandler(change):
    referral_reason_output.clear_output() #the output didn't refersh when selecting a new referral reason rather it will append at the end
    with referral_reason_output:
        if (change.new == ALL):
            display(jurr2data)
        else:
            display(jurr2data[jurr2data.HC_Referral_Reason == change.new])
            
#Step-4 : Bind to the event handler to the dropdown value  
reason4Referral.observe(referral_reason_eventhandler, names='value')

# Step-5: Display the result
display(reason4Referral)
display(referral_reason_output)

3.2. Filtering by referral reason and feedback 

In [ ]:
#RRL fliter by referred reason and feedback  
output = widgets.Output()

reason4Referral = widgets.Dropdown(options =unique_sorted_values(jurr2data.HC_Referral_Reason), description="Ref. Reason")
referredFeedback = widgets.Dropdown(options =unique_sorted_values(jurr2data.Appropriate_Referral_Feedback),description="Feedback: ")


def referral_flitering(referralReason, referralFeedback):
    output.clear_output()

    if (referralReason == ALL) & (referralFeedback == ALL):
        referral_fliter = jurr2data
    elif (referralReason == ALL):
        referral_fliter = jurr2data[jurr2data.Appropriate_Referral_Feedback == referralFeedback]
    elif (referralFeedback == ALL):
        referral_fliter = jurr2data[jurr2data.HC_Referral_Reason == referralReason]
    else:
        referral_fliter = jurr2data[(jurr2data.HC_Referral_Reason == referralReason) & 
                                  (jurr2data.Appropriate_Referral_Feedback == referralFeedback)]
    
    with output:
        display(referral_fliter)
        
def referral_reason_eventhandler(change):
    referral_flitering(change.new, reason4Referral.value)
def referral_feedback_eventhandler(change):
    referral_flitering(reason4Referral.value, change.new)
    
reason4Referral.observe(referral_reason_eventhandler, names='value')
referredFeedback.observe(referral_feedback_eventhandler, names='value')

display(reason4Referral)
display(referredFeedback)
display(output)

4. Visual inspection of the RRL-in clincal data

4.1. Construting histogram for categorical values

In [ ]:
#building Histogram for categorical values or objects 
@interact 
def hist(x=list(jurr2data.select_dtypes('object').columns),
                 #theme=list(cf.themes.THEMES.keys()), 
                 theme=list(cf.getThemes()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    jurr2data[x].iplot(kind='hist', x=x, mode='markers', xTitle='x',
                   yTitle='count', title='Distribution',
                   theme=theme, colorscale=colorscale)

In [ ]:
#building Histogram for categorical values or objects 
@interact 
def hist(x=list(jurr2data.select_dtypes('object').columns),
                 #theme=list(cf.themes.THEMES.keys()), 
                 theme=list(cf.getThemes()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    jurr2data[x].iplot(kind='hist', x=x, mode='markers', xTitle='x',
                   yTitle='count', title='Distribution',
                   theme=theme, colorscale=colorscale)

4.2. Exploring the frequencey or the summary of based on a certain attributes. This helps to visulize the frequencey of the flow of refferal based on the two selected data attributes.

In [ ]:
#Implmentaitng cross tab between two variables 
@interact 
def crosstab(x=list(jurr2data.select_dtypes('object').columns), 
             y=list(jurr2data.select_dtypes('object').columns)[1:],
             theme=list(cf.getThemes()), 
             colorscale=list(cf.colors._scales_names.keys())):
    pd.crosstab(jurr2data[x],jurr2data[y]).plot.barh().legend(loc='center left', bbox_to_anchor=(0.3, 0.5))

4.3. Exploring the frequencey or the summary of based on a certain attributes. This helps to visulize the frequencey of the flow of refferal based on the three selected data attributes.

In [ ]:
# Summary of x= feedback monthly summary with respect to referral reason using hospital RRL and Cardsheet 
#1. try x = referral feedabck, y= month and z = health center referral reason 
#2. try x = referral feedabck, y= month and z = ICPC2 classfication
@interact 
def catplot(x=list(jurr2data.select_dtypes('object').columns), 
            y=list(jurr2data.select_dtypes('number').columns)[1:],
            z=list(jurr2data.select_dtypes('object').columns)[2:],
            theme=list(cf.getThemes()), 
            colorscale=list(cf.colors._scales_names.keys())):
    sns.catplot(x=x, y=y, hue=z,aspect=2,kind="swarm",  data=jurr2data);

### Exploring referral feedback and conclusion

In [ ]:
jurr2data['Appropriate_Referral_Feedback'].value_counts()

In [ ]:
#jurr2data.groupby('Appropriate_Referral_Feedback').mean()
#jurr2data[["Appropriate_Referral_Feedback"]].describe()
jurr2data['Appropriate_Referral_Feedback'].value_counts(normalize=True) * 100

In [ ]:
sns.countplot(x="Appropriate_Referral_Feedback", data=jurr2data)
plt.title('Referred Hospital Referral Feedback and Conclusion')
plt.ylabel("Number of referral cases")
plt.ylabel("Hospital Referral Feedback and Conclusion")

In [ ]:
g=sns.catplot(x="Appropriate_Referral_Feedback", y="Month", hue="HC_Referral_Reason",kind="swarm",aspect=1,  data=jurr2data);
plt.xlabel('Referral Feed back ')
plt.ylabel('Month')
legend = g._legend
legend.set_title("Referral Reason")

In [ ]:
g=sns.catplot(x="Appropriate_Referral_Feedback", y="Month", hue="ICPC2_Classfication",kind="swarm",aspect=1,  data=jurr2data);
plt.xlabel('Referral Feed back ')
plt.ylabel('Month')
legend = g._legend
legend.set_title("Referral Cases(ICPC2 Classfication Format)")

# Part III - Comparing Healthcenter RRL-out and Reffered Hopspital RRL-in and Cardsheet Data

1. Referral Reason: Visualization of healthcenter referral based on RRL-out reason

In [ ]:
#Summary of referral reason distrubtion using health center RRL
rage = hcrrldf.Reason_for_Referral_Trans
xp = rage.value_counts()
plt.title('Referral Registration Logbook', fontsize=14)
plt.xlabel('Referral Reason', fontsize=14)
plt.ylabel('Summary', fontsize=14)
xp.plot(kind='barh')

2. Referral Reason: Visualization based on referred hospital based on RRL-in and cardsheet 

In [ ]:
#Exploring the distrubtion of referral using hospital RRL and Cardsheet
rage = jurr2data.HC_Referral_Reason
xp = rage.value_counts()
plt.title('Referral Registration Logbook', fontsize=14)
plt.xlabel('Referral Reason', fontsize=14)
plt.ylabel('Summary', fontsize=14)
xp.plot(kind='barh')

3. Visualization the RRL-out at the health center relative to RRL-in and card sheet referral reasoning  

In [ ]:
# Compute RRL-out and RRL-in aggreagted Matrix using crosstabs() Method: Categorical Columns
pd.crosstab(index=hcrrldf['Reason_for_Referral_Trans'],columns=jurr2data['HC_Referral_Reason'],
            #colnames=jurr2data['Appropriate_Referral_Trans'],
            margins=True).rename(columns={"RRL_Out": "Count by Reason", "RRL_in": "Count by reason"}).style.background_gradient(cmap='Reds')

In [ ]:
# Compute RRL-out and RRL-in aggreagted Matrix using crosstabs() Method: Categorical Columns
pd.crosstab(index=jurr2data['HC_Referral_Reason'],columns=hcrrldf['Reason_for_Referral_Trans'],
            #colnames=jurr2data['Appropriate_Referral_Trans'],
            margins=True).style.background_gradient(cmap='Reds')

In [ ]:
#Implmentaitng cross tab between two variables the referral reason between RRL and RRL-in
#jurr2data['Appropriate_Referral_Trans']
#Crosstab referral reasoning matrix between Health center RRL-out and referred Hosp RRLin and Cardshet


@interact 
def crosstab(x=widgets.IntSlider(min=0, max=30, step=1, value=7),
             y=widgets.IntSlider(min=0, max=30, step=1, value=7)):
    pd.crosstab(hcrrldf['Reason_for_Referral_Trans'],jurr2data['HC_Referral_Reason']).plot.barh(figsize=(x,y)).legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xlabel("Aggregated Matrix Summary")
    #by Referral Reason:Computed based on RRL-out and RRL-in
    plt.ylabel("Health Center Referral Reason")
    legend = g._legend
    plt.legend(title="Referral Reason at the Referred Hospital")
    legend.set_title("Referral Reason at the Referred Hospital")

In [ ]:
c = pd.crosstab(hcrrldf['Reason_for_Referral_Trans'],jurr2data['HC_Referral_Reason'])
c.plot.barh(figsize=(8,15), width=1)
#.plot.barh(figsize=(10,5)).legend(loc='center left', bbox_to_anchor=(0.5, 0.5))
plt.xlabel("Aggregated Matrix Summary")
#by Referral Reason:Computed based on RRL-out and RRL-in
plt.ylabel("Health Center Referral Reason")
legend = g._legend
plt.legend(title="Referral Reason at the Referred Hospital")
legend.set_title("Referral Reason at the Referred Hospital")

In [ ]:
jurr2data.head(1)

In [ ]:
#Creating a new dataset for comparing the health center dataset and the hospital dataset 

# referral information from the the health center   
hc_referralReason=hcrrldf.Reason_for_Referral_Trans
hc_disease_name = hcrrldf.Type_of_Disease

# referral information from the referred hospital 
hospital_diseaseName = jurr2data.Disease_Name
hospital_referralReason = jurr2data.Referred_Reason_Trans
hospital_feedback = jurr2data.Appropriate_Referral_Trans


In [ ]:
count = jurr2data.Referred_Reason_Trans.value_counts()
count1 = hcrrldf.Reason_for_Referral_Trans.value_counts()


g = sns.PairGrid(hcrrldf, vars=['Age', 'Reason_for_Referral_Trans'],
                 hue='Year', palette='RdBu_r')
g.map(plt.scatter, alpha=0.8)
g.add_legend();

In [ ]:
#Creating a new dataframe from healthcenter referral log book and Hospital RRL and cardsheet

# create a temporaliy copy of df using the selected attributes from the two dataset sources 

# since the dataset collected from the Hopspital RRL is 201 and need to randomly copy 201 records from the source and create comparsiondf
comparisiond201Sample = hcrrldf.sample(201)
comparisiondf = comparisiond201Sample[['Type_of_Disease','Reason_for_Referral_Trans']].copy()
comparisiondf2 = jurr2data[['Disease_Name','Referred_Reason_Trans','Appropriate_Referral_Trans']].copy()


In [ ]:
# concat the selected copy of attributes 
comparisiondf3 = pd.concat([comparisiondf, comparisiondf2], axis=1)

In [ ]:
#The summary of the health center referral summary
countVal = comparisiondf3.Reason_for_Referral_Trans
xp = countVal.value_counts()
plt.title('RRL Viz', fontsize=14)
plt.xlabel('Satus', fontsize=14)
plt.ylabel('Summary', fontsize=14)
xp.plot(kind='bar')

In [ ]:
#The summary of the referral reason at referred hospital RRL Sheet and Cardsheet
countVal = comparisiondf3.Referred_Reason_Trans
xp = countVal.value_counts()
plt.title('RRL Viz', fontsize=14)
plt.xlabel('Satus', fontsize=14)
plt.ylabel('Summary', fontsize=14)
xp.plot(kind='bar')